In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data_paths import *

In [ ]:
df = pd.read_pickle(MERGED)
cosmic_proteins = pd.read_csv(COSMIC_PROTEINS, sep='\t')

In [ ]:
cosmic_proteins.head(1)

In [ ]:
most_studied_proteins = ['Cytochrome P450 3A4', 'Epidermal growth factor receptor', 'Proto-oncogene tyrosine-protein kinase Src', 'Vascular endothelial growth factor receptor 2', 'Adenosine receptor A2a', 'Cytochrome P450 2C9', 'Cytochrome P450 1A2', 'Cytochrome P450 2C19', 'Cytochrome P450 2D6', 'Prostaglandin G/H synthase 1', 'Prostaglandin G/H synthase 2']

### Check if synonyms could be used

In [ ]:
arr = cosmic_proteins['Gene synonym'].apply(lambda x: np.array([y.strip() for y in str(x).split(',')]))
all_elems = np.concatenate(arr) 

# Check isin for each element in the array (careful of double lists)
df['target_name'].isin(all_elems).sum(), df['drugbank_protein_name'].isin(all_elems).sum()

In [ ]:
cosmic_proteins['Uniprot']

In [ ]:
# Matching by drugbank_protein_name
nb_bind_in_cosmic_bn = df['drugbank_protein_name'].isin(cosmic_proteins['Gene']).sum()
nb_cosmic_in_bind_bn = cosmic_proteins['Gene'].isin(df['drugbank_protein_name']).sum()
nb_bind_in_cosmic_by_targetname_bn = df['target_name'].isin(cosmic_proteins['Gene']).sum()

# Matching by uniprot
nb_bind_in_cosmic_bu = df['swissprot_protein_id'].dropna().isin(cosmic_proteins['Uniprot']).sum()
nb_cosmic_in_bind_bu = cosmic_proteins['Uniprot'].isin(df['swissprot_protein_id'].dropna()).sum()

print(f"Number of bind proteins in cosmic by drugbank_protein_name: {nb_bind_in_cosmic_bn}")
print(f"Number of cosmic proteins in bind by drugbank_protein_name: {nb_cosmic_in_bind_bn}")
print()
print(f"Number of bind proteins in cosmic by target_name: {nb_bind_in_cosmic_by_targetname_bn}")
print()
print(f"Number of bind proteins in cosmic by uniprot: {nb_bind_in_cosmic_bu}")
print(f"Number of cosmic proteins in bind by uniprot: {nb_cosmic_in_bind_bu}")

### Cancer related proteins

In [ ]:
# cosmic in most studied proteins
cosmic_proteins['Gene'].isin(most_studied_proteins).sum()

In [ ]:
bind_cancer_proteins_by_name = df[df['drugbank_protein_name'].isin(cosmic_proteins['Gene'])]
bind_cancer_proteins_by_uniprot = df[df['swissprot_protein_id'].isin(cosmic_proteins['Uniprot'])]

# Add most studied proteins
bind_cancer_proteins_by_name = pd.concat([bind_cancer_proteins_by_name, df[df['target_name'].isin(most_studied_proteins)]])

bcpbn_set = set(bind_cancer_proteins_by_name['target_name'])
bcpbu_set = set(bind_cancer_proteins_by_uniprot['target_name'])

print(f"Number of cancer proteins in bind by drugbank_protein_name: {len(bcpbn_set)}")
print(f"Number of cancer proteins in bind by uniprot: {len(bcpbu_set)}")
print(f"Difference between the two sets:")
print(f"  - {len(bcpbn_set - bcpbu_set)} proteins in drugbank_protein_name but not in uniprot")
print(f"  - {len(bcpbu_set - bcpbn_set)} proteins in uniprot but not in drugbank_protein_name")

In [ ]:
cancer_proteins_namey_not_uniprot = df[df['target_name'].isin(bcpbn_set - bcpbu_set)]
cancer_proteins_namey_not_uniprot[['drugbank_protein_name', 'target_name', 'swissprot_protein_id']]

In [ ]:
cosmic_proteins[cosmic_proteins['Gene'] == 'EGFR'][['Gene', 'Gene synonym', 'Uniprot']]

In [ ]:
# Combine the two sets (even though Uniprot is not the same)
cancer_related_proteins = pd.concat([bind_cancer_proteins_by_name, bind_cancer_proteins_by_uniprot]).drop_duplicates()
cancer_related_proteins_df = df[df['target_name'].isin(cancer_related_proteins['target_name'])]

print(f"Number of pairs of cancer proteins with their ligand in bind: {len(cancer_related_proteins_df)}")
print(f"Number of unique cancer proteins: {len(cancer_related_proteins['target_name'].unique())}")
print(f"  - {len(bind_cancer_proteins_by_name['target_name'].unique())} by drugbank_protein_name")
print(f"  - {len(bind_cancer_proteins_by_uniprot['target_name'].unique())} by uniprot")

### Ligands related to these proteins

In [ ]:
ligands_related_to_cancer_proteins = cancer_related_proteins_df.dropna(subset=['ligand_name'])

print(f"Number of pairs of ligands related to cancer proteins (keep only the ones for which we have a name): {len(ligands_related_to_cancer_proteins)}")
print(f"Number of unique ligands related to cancer proteins: {len(set(ligands_related_to_cancer_proteins['ligand_name']))}")
print(f"Number of cancer proteins that matched to a ligand: {len(set(ligands_related_to_cancer_proteins['target_name']))} out of {len(cancer_related_proteins['target_name'].unique())}")

In [ ]:
# Keep only the ligands that are in DrugBank 
drugs_related_to_cancer_proteins = ligands_related_to_cancer_proteins.dropna(subset='drugbank_drug_class_superclass')

print(f"Number of pairs of drugs related to cancer proteins: {len(drugs_related_to_cancer_proteins)}")
print(f"Number of unique drugs related to cancer proteins: {len(drugs_related_to_cancer_proteins['drugbank_drug_name'].unique())}")
print(f"Number of cancer proteins that matched to a drug: {len(set(drugs_related_to_cancer_proteins['target_name']))} out of {len(cancer_related_proteins['target_name'].unique())}")

### All proteins related to these ligands

In [ ]:
all_prots_related_to_cancer_ligands = df[df['ligand_name'].isin(ligands_related_to_cancer_proteins['ligand_name'].unique())]
all_prots_related_to_cancer_drugs = df[df['drugbank_drug_name'].isin(drugs_related_to_cancer_proteins['drugbank_drug_name'].unique())]

print(f"Number of unique proteins related to cancer ligands: {len(all_prots_related_to_cancer_ligands['target_name'].unique())}")
print(f"Number of unique proteins related to cancer drugs: {len(all_prots_related_to_cancer_drugs['target_name'].unique())}")

### Differentiate between secondary effects and drugs

In [ ]:
direct_prots_related_target_names = cancer_related_proteins['target_name'].unique()

direct_effect_prots = all_prots_related_to_cancer_drugs[all_prots_related_to_cancer_drugs['target_name'].isin(direct_prots_related_target_names)]
secondary_effect_prots = all_prots_related_to_cancer_drugs[~all_prots_related_to_cancer_drugs['target_name'].isin(direct_prots_related_target_names)]

ligands_direct_effect_prots = all_prots_related_to_cancer_ligands[all_prots_related_to_cancer_ligands['target_name'].isin(direct_prots_related_target_names)]
ligands_secondary_effect_prots = all_prots_related_to_cancer_ligands[~all_prots_related_to_cancer_ligands['target_name'].isin(direct_prots_related_target_names)]

print(f"Number of proteins directly targeted by drugs: {len(direct_effect_prots['target_name'].unique())}")
print(f"Number of proteins representing secondary effect of drugs: {len(secondary_effect_prots['target_name'].unique())}")
print()
print(f"Number of proteins directly targeted by ligands: {len(ligands_direct_effect_prots['target_name'].unique())}")
print(f"Number of proteins representing secondary effect of ligands: {len(ligands_secondary_effect_prots['target_name'].unique())}")

Note that cancer related proteins has 146 distinct prots, but as we keep only the ones that are linked to a ligand in bindingdb, we end up using only 66 (61 for drugs) out of those 146 prots. So in the end, the difference between number of proteins directly targeted by ligands and the number of proteins that are related to cancer is only because of this:
```	python
    ligands_related_to_cancer_proteins = cancer_related_proteins_df.dropna(subset=['drugbank_drug_name'])
```
In summary, 
- we have 146 cancer prots
- they link to 405 ligands (ligands related to cancer prots), which themselves only link to 66 of the cancer prots (61 for drugs)
- these 405 ligands link to all prots which around 1651 prots
- out of all these prots, as we've seen 66 are cancer related (61 for drugs), and 1504 are likely proteins on which the drug has secondary effects

The difference between the 146 cancer prots and the 66 cancer prots is really because only a subset of the cancer prots can be used for our analysis. And this subset, as explained above, is given by the dropna on drugbank_drug_name which is the name of the ligand.

In [ ]:
ligands_related_to_cancer_proteins

In [ ]:
ligands_related_to_cancer_proteins.to_pickle("data/ligands_related_to_cancer_proteins.pkl")

In [ ]:
direct_effect_prots['drugbank_drug_name'].isin(all_prots_related_to_cancer_drugs['drugbank_drug_name']).sum()

In [ ]:

# Create a list of nodes
nodes = []
node_id_map = {}  # To map node names to unique ids
categories = [{'name': 'Possible Cancer Drug'}, {'name': 'Cancer Protein'}, {'name': 'Secondary Effect Protein'}]

# Process ligand nodes
ligand_nodes = all_prots_related_to_cancer_drugs['drugbank_drug_name'].dropna().unique()
for idx, ligand in enumerate(ligand_nodes):
    ligand = str(ligand)
    node_id_map[ligand] = idx
    node = {
        'id': str(idx),
        'Label': ligand,
        'category': 0,  # Index of 'ligand' in categories
        'value': 1  # You can set this to degree or other measure
    }
    nodes.append(node)

# Process protein nodes
cancer_protein_nodes = direct_effect_prots['target_name'].unique()
for idx, protein in enumerate(cancer_protein_nodes, start=len(node_id_map)):
    node_id_map[protein] = idx
    node = {
        'id': str(idx),
        'Label': protein,
        'category': 1,  # Index of 'protein' in categories
        'value': 1  # You can set this to degree or other measure
    }
    nodes.append(node)

secondary_protein_nodes = secondary_effect_prots['target_name'].unique()
for idx, protein in enumerate(secondary_protein_nodes, start=len(node_id_map)):
    node_id_map[protein] = idx
    node = {
        'id': str(idx),
        'Label': protein,
        'category': 2,  # Index of 'protein' in categories
        'value': 1  # You can set this to degree or other measure
    }
    nodes.append(node)


In [ ]:
# Create a list of edges
edges = []
for _, row in direct_effect_prots.iterrows():
    source_id = node_id_map[row['drugbank_drug_name']]
    target_id = node_id_map[row['target_name']]
    strength = row['ec50'] if pd.isna(row['ic50']) else row['ic50']  # Use ec50 if ic50 is NaN
    
    if pd.isna(strength) or strength < 1e-8 or np.log(strength) < 1e-5:
        continue

    edge = {
        'source': str(source_id),
        'target': str(target_id),
        'Weight': np.log(strength)  # Log-transform the value to make it more visually appealing
    }
    edges.append(edge)

for _, row in secondary_effect_prots.iterrows():
    source_id = node_id_map[row['drugbank_drug_name']]
    target_id = node_id_map[row['target_name']]
    strength = row['ec50'] if pd.isna(row['ic50']) else row['ic50']  # Use ec50 if ic50 is NaN

    if pd.isna(strength) or strength < 1e-8 or np.log(strength) < 1e-5:
        continue

    edge = {
        'source': str(source_id),
        'target': str(target_id),
        'Weight': np.log(strength)
    }
    edges.append(edge)

# Create the graph data
graph_data = {
    'nodes': nodes,
    'links': edges,
    'categories': categories
}

In [ ]:
# Store as csv the nodes and the edges
nodes_df = pd.DataFrame(nodes)
edges_df = pd.DataFrame(edges)

nodes_df.to_csv('graph_data/nodes.csv', index=False)
edges_df.to_csv('graph_data/edges.csv', index=False)

In [ ]:
from pyecharts.charts import Graph
from pyecharts import options as opts


graph = Graph(init_opts=opts.InitOpts(width="100%", height="700px"))
graph.add(
    "",
    nodes,
    edges,
    categories=categories,
    layout="force",
    edge_length=[50, 200],
    repulsion=100,
    linestyle_opts=opts.LineStyleOpts(width=0.5, opacity=0.5),
)
graph.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=False,  # Hide labels by default
        position="right",
        formatter="{b}",  # Use node name as the label
        #show=False,  # Ensure labels are not rendered statically
    )
)

graph.set_global_opts(
    title_opts=opts.TitleOpts(title="Cancer Related Ligand-Protein Interaction Network"),
)

graph.render("graph_data/graph.html")


In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Extract node and edge data
nodes = graph_data['nodes']
edges = graph_data['links']
categories = graph_data['categories']

# Create an igraph graph
g = ig.Graph(directed=False)  # Set directed=True if your data is directional
g.add_vertices(len(nodes))

# Add vertex attributes
for i, node in enumerate(nodes):
    g.vs[i]['name'] = node['name']
    g.vs[i]['category'] = categories[node['category']]['name']

# Add edges
edge_list = [(int(e['source']), int(e['target'])) for e in edges]
g.add_edges(edge_list)

# Add edge attributes
if 'value' in edges[0]:
    for i, e in enumerate(edges):
        g.es[i]['ic50'] = e['value']

# Layout the graph (try 'kk', 'fr', or 'graphopt' for large graphs)
layout = g.layout('fr')

# Assign colors to categories
category_names = [cat['name'] for cat in categories]
# Generate distinct colors for each category - you can customize this palette
cmap = ListedColormap(["#1f77b4", "#ff7f0e", "#2ca02c"])
category_color_map = {cat: cmap(i) for i, cat in enumerate(category_names)}
node_colors = [category_color_map[g.vs[i]['category']] for i in range(g.vcount())]

# Adjust node size if needed
node_size = 8



In [ ]:

# Scale edge width by ic50 if present
if 'ic50' in g.es.attributes():
    # Adjust scaling as needed
    # For example, you could normalize values or apply a log scale if necessary
    edge_widths = [np.log(0.5 + 2*float(val) if val is not pd.NA else 0.5) for val in g.es['ic50']]
else:
    edge_widths = [1]*g.ecount()

# Normalize edge width
edge_widths = np.array(edge_widths)
edge_widths = (edge_widths - edge_widths.min()) / (edge_widths.max() - edge_widths.min())

# If the graph is very large, consider not drawing labels or use a subset
draw_labels = False  # Set to True if you want labels (may be messy in large graphs)
vertex_labels = g.vs['name'] if draw_labels else None

# Create a figure
fig, ax = plt.subplots(figsize=(15, 10))

# Plot the graph
ig.plot(
    g,
    target=ax,
    layout=layout,
    vertex_size=node_size,
    vertex_color=node_colors,
    vertex_label=vertex_labels,
    vertex_label_size=8,
    edge_width=edge_widths,
    edge_color="gray",
    bbox=(1000, 800),  # Control size in pixels
    margin=50
)

# Create a legend for categories
handles = [plt.Line2D([], [], marker='o', color=cmap(i), linestyle='None', markersize=10, label=name) 
           for i, name in enumerate(category_names)]
ax.legend(handles=handles, title="Categories", loc='best')

plt.tight_layout()
plt.show()